In [2]:
# load dependancies
import pandas as pd

In [3]:
# read in csv to pandas dataframe
raw_data = pd.read_csv('2022-23_data_sa_crime.csv')

In [4]:
df = raw_data.copy()

# drop columns that are not needed
df.drop(['Offence Level 3 Description'], axis=1, inplace=True)

# change reported date to ISO format (YYYY-MM-DD)
df['Reported Date'] = pd.to_datetime(df['Reported Date'], format='%d/%m/%Y')

# rename columns
df.columns = ['Date', 'Suburb', 'Postcode', 'Offence Level 1 Description', 'Offence Level 2 Description', 'Offence Count']

In [5]:
# string of data with postcode, suburb, ditance from cbd, first line is not a header
radius50km = pd.read_csv('radius50kmcbd.csv', header=None)

# split data into columns, delimiter is ","
radius50km = radius50km[0].str.split(',', expand=True)

# rename columns
radius50km.columns = ['Postcode', 'Location', 'Distance from CBD (km)']

# drop location column
radius50km.drop(['Location'], axis=1, inplace=True)

# aggregate all rows by postcode, average the distance from cbd if not 0
radius50km = radius50km.groupby(['Postcode'], as_index=False).agg(
    {'Distance from CBD (km)': lambda x: x.astype(float).mean(skipna=True)}
)

In [6]:
# drop all rows from df if the postcode is not in the radius50km dataframe
df_50kmradius = df[df['Postcode'].isin(radius50km['Postcode'])]

# add the distance from cbd column to df_50kmradius
df_50kmradius = pd.merge(df_50kmradius, radius50km, on='Postcode')

In [7]:
# summarise offences by level 2 description
df_level2 = df_50kmradius.groupby(['Date', 'Suburb', 'Offence Level 2 Description'], as_index=False).agg(
    {'Postcode': 'first', 
     'Distance from CBD (km)': 'first',
     'Offence Level 1 Description': 'first', 
     'Offence Level 2 Description': 'first',
     'Offence Count': 'sum'
     }
)                  

In [8]:
# summarise offences by level 1 description
df_level1 = df_50kmradius.groupby(['Date', 'Suburb', 'Offence Level 1 Description'], as_index=False).agg(
    {'Postcode': 'first', 
     'Distance from CBD (km)': 'first',
     'Offence Level 1 Description': 'first', 
     'Offence Count': 'sum',}
)    

In [9]:
print(f"length of df {len(df)}")
print(f"length of df with only suburbs in 50km radius {len(df_50kmradius)}")
print(f"length of df_level2 {len(df_level2)}")
print(f"length of df_level1 {len(df_level1)}")

length of df 97078
length of df with only suburbs in 50km radius 77456
length of df_level2 68046
length of df_level1 54820


In [10]:
# export all to csv
# df.to_csv('all_data_clean.csv', index=False)
# df_50kmradius.to_csv('50kmradius_data_clean.csv', index=False)
# df_level2.to_csv('level2_data_clean.csv', index=False)
# df_level1.to_csv('level1_data_clean.csv', index=False)

In [11]:
# dictionary of shorter terms
short_terms = {
    'Offence Level 1 Description': {
        'OFFENCES AGAINST THE PERSON': 'PERSON',
        'OFFENCES AGAINST PROPERTY': 'PROPERTY',
    },
    'Offence Level 2 Description': {
        'FRAUD DECEPTION AND RELATED OFFENCES': 'FRAUD',
        'PROPERTY DAMAGE AND ENVIRONMENTAL': 'PROPERTY DAMAGE',
        'SERIOUS CRIMINAL TRESPASS': 'TRESPASS',
        'THEFT AND RELATED OFFENCES': 'THEFT',
        'ACTS INTENDED TO CAUSE INJURY': 'INJURY',
        'OTHER OFFENCES AGAINST THE PERSON': 'OTHER',
        'ROBBERY AND RELATED OFFENCES': 'ROBBERY',
        'SEXUAL ASSAULT AND RELATED OFFENCES': 'SEXUAL ASSAULT',
        'HOMICIDE AND RELATED OFFENCES': 'HOMICIDE'
    }
}

In [12]:
# final dataframe is df_50kmradius with short terms
df_final = df_50kmradius.copy()
df_final.replace(short_terms, inplace=True)

In [13]:
df_final.to_csv('final_data_clean.csv', index=False)

In [14]:
%pip install --upgrade sqlalchemy

Note: you may need to restart the kernel to use updated packages.


In [17]:
# using sqlite and sqlalchemy, export df_final to a sql database
from sqlalchemy import create_engine
engine = create_engine('sqlite:///crime_data.db', echo=True)
sqlite_connection = engine.connect()
sqlite_table = "crime_data"
df_final.to_sql(sqlite_table, sqlite_connection, if_exists='replace')

# SELECT first ten entries from the database
sqlite_query = 'SELECT * FROM crime_data LIMIT 10'
df_sql = pd.read_sql_query(sqlite_query, sqlite_connection)

2024-01-22 19:37:16,684 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-01-22 19:37:16,691 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("crime_data")
2024-01-22 19:37:16,691 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-01-22 19:37:16,693 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("crime_data")
2024-01-22 19:37:16,693 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-01-22 19:37:16,694 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2024-01-22 19:37:16,694 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-01-22 19:37:16,695 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='view' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2024-01-22 19:37:16,697 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-01-22 19:37:16,705 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("crime_data")
2024-01-22 19:37:16,713 INFO sqlalchemy.engine.Engine [raw sql] ()
202

In [18]:
df_sql

,index,Date,Suburb,Postcode,Offence Level 1 Description,Offence Level 2 Description,Offence Count,Distance from CBD (km)
0,0,2022-07-01 00:00:00.000000,ADELAIDE,5000,PROPERTY,FRAUD,1,0.0
1,1,2022-07-01 00:00:00.000000,ADELAIDE,5000,PROPERTY,PROPERTY DAMAGE,1,0.0
2,2,2022-07-01 00:00:00.000000,ADELAIDE,5000,PROPERTY,PROPERTY DAMAGE,2,0.0
3,3,2022-07-01 00:00:00.000000,ADELAIDE,5000,PROPERTY,TRESPASS,1,0.0
4,4,2022-07-01 00:00:00.000000,ADELAIDE,5000,PROPERTY,THEFT,2,0.0
5,5,2022-07-01 00:00:00.000000,ADELAIDE,5000,PROPERTY,THEFT,8,0.0
6,6,2022-07-01 00:00:00.000000,ADELAIDE,5000,PERSON,INJURY,1,0.0
7,7,2022-07-02 00:00:00.000000,ADELAIDE,5000,PROPERTY,TRESPASS,1,0.0
8,8,2022-07-02 00:00:00.000000,ADELAIDE,5000,PROPERTY,THEFT,2,0.0
9,9,2022-07-02 00:00:00.000000,ADELAIDE,5000,PROPERTY,THEFT,7,0.0
